In [ ]:
from __future__ import absolute_import, division, print_function

import dolfin as dl
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import hippylib as hp

import logging
logging.getLogger('FFC').setLevel(logging.WARNING)
logging.getLogger('UFL').setLevel(logging.WARNING)
dl.set_log_active(False)
import ipywidgets as wd
np.random.seed(seed=1)

In [ ]:
def init_problem(num_x=20, num_y=20):
    ndim = 2
    mesh = dl.UnitSquareMesh(num_x, num_y)
    Vh2 = dl.FunctionSpace(mesh, 'Lagrange', 2)
    Vh1 = dl.FunctionSpace(mesh, 'Lagrange', 1)
    Vh = [Vh2, Vh1, Vh2]
    print( "Number of dofs: STATE={0}, PARAMETER={1}, ADJOINT={2}".format(
        Vh[hp.STATE].dim(), Vh[hp.PARAMETER].dim(), Vh[hp.ADJOINT].dim()) )
    return Vh

Vh40 = init_problem(40,40)
Vh20 = init_problem(20,20)
Vh10 = init_problem(10,10)

---
# Treat the forcing as certain and the material properties as uncertain

In [ ]:
def make_pde_for_hp_m(Vh, omega_const=0.5, f_const=1.0, g_const=0.0):
    f = dl.Constant(f_const)
    g = dl.Constant(g_const)
    omega = dl.Constant(omega_const)

    def pde_varf(u,m,p):
        return -dl.inner(dl.nabla_grad(u), dl.nabla_grad(p))*dl.dx + omega*omega*dl.exp(2.0*m)*dl.inner(u, p)*dl.dx + f*p*dl.dx  + g*p*dl.ds
    
    pde = hp.PDEVariationalProblem(Vh, pde_varf, [], [], is_fwd_linear=True)
    return pde


In [ ]:
def generate_prior_and_true_model(Vh, gamma=1.0, delta=8.0, seed=1):
#     anis_diff = dl.Expression(hp.code_AnisTensor2D, degree=1)
#     anis_diff.theta0 = 2.
#     anis_diff.theta1 = .5
#     anis_diff.alpha = math.pi/4
#     prior = hp.BiLaplacianPrior(Vh[hp.PARAMETER], gamma, delta, anis_diff)

    prior = hp.BiLaplacianPrior(Vh[hp.PARAMETER], gamma, delta)
    prior.mean = dl.interpolate(dl.Constant(1), Vh[hp.PARAMETER]).vector()
    # This function generates a random field with a prescribed anisotropic covariance function.
    def true_model(prior):
        noise = dl.Vector()
        prior.init_vector(noise, "noise")
        _world_rank = dl.MPI.rank(dl.mpi_comm_world())
        _world_size = dl.MPI.rank(dl.mpi_comm_world())
        myRandom = hp.Random(_world_rank, _world_size, seed=seed)
        
        myRandom.normal(1., noise) # variance 1 noise

        mtrue = dl.Vector()
        prior.init_vector(mtrue, 0)

        prior.sample(noise, mtrue)
        return mtrue

    return prior, true_model

def visualize_true_field(Vh, gamma=1.0, delta=8.0, seed=1, cbar = 0.2):   
    print("Prior regularization: (delta_x - gamma*Laplacian)^order: delta={0}, gamma={1}, order={2}".format(delta, gamma, -2))    

    plt.figure(figsize=(15,10))
    for i in range(1,10):
        if(i != 5):
            ss_loc = int(330+i)
            prior, true_model = generate_prior_and_true_model(Vh, gamma, delta, seed=i)
            mtrue = true_model(prior) # generate realization
            hp.nb.plot(dl.Function(Vh[hp.PARAMETER], mtrue), subplot_loc=ss_loc, 
                       mytitle="Realization", vmin=1-cbar, vmax=1+cbar)
    
    hp.nb.plot(dl.Function(Vh[hp.PARAMETER], prior.mean), subplot_loc=335, 
               mytitle="Mean", vmin=1-cbar, vmax=1+cbar)
    plt.show()
    return None



## Play around with this widget to define some true parameter

$\sqrt(\gamma/\delta)$ is proportional to the correlation length in the field. 

In [ ]:
Prior_Wid = wd.interact_manual(visualize_true_field, 
            Vh = wd.fixed(Vh10),
            gamma = wd.FloatSlider(value=1.0, min=0.25, max=10.0, step=0.25, continuous_update=False),
            delta = wd.FloatSlider(value=8.0, min=0.25, max=10.0, step=0.25, continuous_update=False),
            seed = wd.IntSlider(value=1, min=1, max=100, step=1, continuous_update=False),
            cbar = wd.FloatSlider(value=0.25, min=0.05, max=0.5, step=0.05, continuous_update=False))


## Run cell below with values you decided on to set them. 

In [ ]:
def solve_problem(Vh, omega_const, f_const, g_const, gamma, delta, num_sensors = 100, rel_noise = 0.005, max_its=20, seed=1, cbar=0.25):
    prior, true_model = generate_prior_and_true_model(Vh, gamma, delta, seed)
    mtrue = true_model(prior) # generate realization
    
#     np.random.seed(1)
    sensor_locs = np.random.uniform(0.05,0.95, [num_sensors, 2] )
    print( "Number of observation points: {0} with relative noise {1}".format(num_sensors, rel_noise) )
    misfit = hp.PointwiseStateObservation(Vh[hp.STATE], sensor_locs)
    
    # Instantiate the PDE Object for hippylib
    pde = make_pde_for_hp_m(Vh, omega_const, f_const, g_const)
    utrue = pde.generate_state()
    x = [utrue, mtrue, None]
    pde.solveFwd(x[hp.STATE], x, 1e-9)
    misfit.B.mult(x[hp.STATE], misfit.d)
    MAX = misfit.d.norm("linf")
    
    noise_std_dev = rel_noise * MAX
    hp.parRandom.normal_perturb(noise_std_dev, misfit.d)
    misfit.noise_variance = noise_std_dev*noise_std_dev
    
    # Instantiate the Model Object for hippylib
    model = hp.Model(pde, prior, misfit)
    
    # Solver
    m = prior.mean.copy()
    solver = hp.ReducedSpaceNewtonCG(model) # CHOOSE SOLVER
    solver.parameters["rel_tolerance"] = 1e-6
    solver.parameters["abs_tolerance"] = 1e-12
    solver.parameters["max_iter"]      = max_its
    solver.parameters["inner_rel_tolerance"] = 1e-15
    solver.parameters["GN_iter"] = 5
    solver.parameters["globalization"] = "LS"
    solver.parameters["LS"]["c_armijo"] = 1e-4
    solver.parameters["print_level"] = -1
    
    x = solver.solve([None, m, None])
    
    if solver.converged:
        print( "\nConverged in ", solver.it, " iterations.")
    else:
        print( "\nNot Converged")

    print( "Termination reason: ", solver.termination_reasons[solver.reason] )
    print( "Final gradient norm: ", solver.final_grad_norm )
    print( "Final cost: ", solver.final_cost )

    # Plotting at end
    plt.figure(figsize=(15,10))
    vmax = max( utrue.max(), misfit.d.max() )
    vmin = min( utrue.min(), misfit.d.min() )
    
    hp.nb.plot(dl.Function(Vh[hp.PARAMETER], mtrue), subplot_loc=331, 
               mytitle="True Parameter", vmin=1-cbar, vmax=1+cbar)
    hp.nb.plot(dl.Function(Vh[hp.STATE], utrue), 
               mytitle="True State", subplot_loc=332, vmin=vmin, vmax=vmax)
    hp.nb.plot_pts(sensor_locs, misfit.d, 
                   mytitle="Observations", subplot_loc=333, vmin=vmin, vmax=vmax)
    
    hp.nb.plot(dl.Function(Vh[hp.PARAMETER], x[hp.PARAMETER]), 
               mytitle="Estimated Parameter", subplot_loc=334, vmin=1-cbar, vmax=1+cbar)
    hp.nb.plot(dl.Function(Vh[hp.STATE], x[hp.STATE]),
               mytitle="Estimated State", subplot_loc=335, vmin=vmin, vmax=vmax)

    plt.show()
    

    

#     m0 = dl.interpolate(dl.Expression("sin(x[0])", degree=5), Vh[hp.PARAMETER])
#     _ = hp.modelVerify(model, m0.vector(), 1e-12)
    

In [ ]:
W_omega_const = wd.FloatSlider(value=1.0, min=0.05, max=5.0, step=0.05, continuous_update=False)
W_f_const = wd.FloatSlider(value=0.0, min=0, max=10.0, step=0.25, continuous_update=False)
W_g_const = wd.FloatSlider(value=1.0, min=0, max=10.0, step=0.25, continuous_update=False)
W_gamma = wd.FloatSlider(value=1.0, min=0.25, max=10.0, step=0.25, continuous_update=False)
W_delta = wd.FloatSlider(value=8.0, min=0.25, max=10.0, step=0.25, continuous_update=False)
W_num_sensors = wd.IntSlider(value=25, min=1, max=500, continuous_update=False)
W_rel_noise = wd.SelectionSlider(value=0.005, options=[("%g"%i,i) for i in [i*10**-4 for i in range(1,101)]], continuous_update=False)                  
W_max_its = wd.IntSlider(value=20, min=1, max=100, continuous_update=False)
W_seed = wd.IntSlider(value=1, min=1, max=100, step=1, continuous_update=False)
W_cbar = wd.FloatSlider(value=0.25, min=0.05, max=0.5, step=0.05, continuous_update=False)

In [ ]:
# gui = wd.HBox([wd.VBox([W_omega_const, W_f_const, W_g_const]), 
#                wd.VBox([W_var, W_gamma, W_delta]), 
#                wd.VBox([W_num_sensors, W_rel_noise, W_max_its, W_cbar]) ])
keydict = {'omega_const': W_omega_const, 
           'f_const': W_f_const, 
           'g_const': W_g_const, 
           'gamma': W_gamma, 
           'delta': W_delta, 
           'num_sensors': W_num_sensors, 
           'rel_noise': W_rel_noise, 
           'max_its': W_max_its, 
           'seed': W_seed,
           'cbar': W_cbar}


In [ ]:
wid_out = wd.interact_manual(solve_problem, Vh = wd.fixed(Vh10), **keydict)

---
# Treat the material properties as certain and the location of a forcing as uncertain

In [ ]:
%%bash 
git commit -am 'updated files'
git status